In [2]:
import pandas as pd

In [3]:
dia_3_df = pd.read_csv('./dia_3.csv')
dia_t_df = pd.read_csv('./dia_t.csv')
diagn_title_df = pd.read_csv('./diagn_title.csv')
diffsydiw_df = pd.read_csv('./diffsydiw.csv')
sym_3_df = pd.read_csv('./sym_3.csv')
dia_dis_matrix_df = pd.read_csv('./sym_dis_matrix.csv')
sym_t_df = pd.read_csv('./sym_t.csv')
symtoms2_df = pd.read_csv('./symptoms2.csv')


# データの統合
この例では、症例（sym_3_df, sym_t_df）と診断（dia_3_df, dia_t_df, diagn_title_df）のデータを統合します。
症状と診断の関連データ（diffsydiw_df）も結合します。


症状データの統合

In [5]:
symptoms_df = pd.merge(sym_3_df, sym_t_df, left_on='_id', right_on='syd', how='inner')


dia_3.csv の '_id' 列を整数型に変換

In [14]:
dia_3_df['_id'] = pd.to_numeric(dia_3_df['_id'], errors='coerce')
dia_t_df['did'] = pd.to_numeric(dia_t_df['did'], errors='coerce')

変換できなかった行を削除

In [18]:
dia_3_df.dropna(subset=['_id'], inplace=True)
dia_t_df.dropna(subset=['did'], inplace=True)

診断データの統合

In [15]:
diagnosis_df = pd.merge(dia_3_df, dia_t_df, left_on='_id', right_on='did', how='inner')
diagnosis_df = pd.merge(diagnosis_df, diagn_title_df, left_on='_id', right_on='id', how='inner')

症状と診断の関連データの統合

In [16]:
symptom_diagnosis_relation = pd.merge(diffsydiw_df, symptoms_df, left_on='syd', right_on='syd', how='inner')
symptom_diagnosis_relation = pd.merge(symptom_diagnosis_relation, diagnosis_df, left_on='did', right_on='did', how='inner')

統合したデータの確認

In [17]:
symptom_diagnosis_relation.head()

,syd,did,wei,_id_x,symptom_x,symptom_y,_id_y,diagnose_x,diagnose_y,id,title
0,1.0,163.0,2.0,1,Upper abdominal pain,Upper abdominal pain,163.0,Cholecystitis inflammation of the gallbladder,Cholecystitis inflammation of the gallbladder,163.0,Cholecystitis inflammation of the gallbladder
1,2.0,163.0,1.0,2,Lower abdominal pain,Lower abdominal pain,163.0,Cholecystitis inflammation of the gallbladder,Cholecystitis inflammation of the gallbladder,163.0,Cholecystitis inflammation of the gallbladder
2,7.0,163.0,1.0,7,Back ache or pain,Back ache or pain,163.0,Cholecystitis inflammation of the gallbladder,Cholecystitis inflammation of the gallbladder,163.0,Cholecystitis inflammation of the gallbladder
3,29.0,163.0,2.0,29,Fever,Fever,163.0,Cholecystitis inflammation of the gallbladder,Cholecystitis inflammation of the gallbladder,163.0,Cholecystitis inflammation of the gallbladder
4,32.0,163.0,1.0,32,Flank pain,Flank pain,163.0,Cholecystitis inflammation of the gallbladder,Cholecystitis inflammation of the gallbladder,163.0,Cholecystitis inflammation of the gallbladder


# 初期診断の提案を行う関数

In [20]:
def initial_diagnosis_suggestion(symptom, data, top_n=5):
    """初期診断を提案する関数"""
    filtered_data = data[data['symptom_x'] == symptom]
    diagnoses = filtered_data.groupby('diagnose_x')['wei'].sum().reset_index()
    top_diagnoses = diagnoses.sort_values(by='wei', ascending=False).head(top_n)
    return top_diagnoses['diagnose_x'].tolist()

# 診断の絞り込みを行う関数

In [25]:
def refine_diagnosis(initial_diagnoses, additional_symptom, data, top_n=1):
    """診断の絞り込みを行う関数"""
    filtered_data = data[data['symptom_x'] == additional_symptom]
    filtered_data = filtered_data[filtered_data['diagnose_x'].isin(initial_diagnoses)]
    refined_diagnoses = filtered_data.groupby('diagnose_x')['wei'].sum().reset_index()
    top_refined_diagnoses = refined_diagnoses.sort_values(by='wei', ascending=False).head(top_n)
    return top_refined_diagnoses['diagnose_x'].tolist()

例：初期症状として「Back ache or pain」が選択され、追加症状として「Fever」が提供された場合の処理

In [26]:
selected_symptom = "Back ache or pain"
additional_symptom = "Fever"

In [27]:
initial_suggestions = initial_diagnosis_suggestion(selected_symptom, symptom_diagnosis_relation)
print("Initial Diagnosis Suggestions:", initial_suggestions)

Initial Diagnosis Suggestions: ['Cholecystitis\x0binflammation of the gallbladder', 'Cholelithiasis\x0bgallstones', 'Lymphoma\x0blymph node cancer', 'Multiple sclerosis\x0bMS']


In [28]:
refined_suggestions = refine_diagnosis(initial_suggestions, additional_symptom, symptom_diagnosis_relation)
print("Refined Diagnosis Suggestions:", refined_suggestions)

Refined Diagnosis Suggestions: ['Cholecystitis\x0binflammation of the gallbladder']
